# Intro to eppy_tutorial
----
#####  Eppy is a scripting language to Energyplus. Eppy puts the idd and idf file into a python data structure, and uses the power of python to edit idf files. 
##### This tutorial is intended to help you take the first step in using eppy to access idf files and run Energyplus. The demo file is a quick introduction of some tasks that eppy could achive:
* read an idf file with eppy
* access idf objects: **material, consatruction, buidling surfaces, building and zone**
    * view these idf objects
    * add new idf objects
    * modify idf objects
* check range with **for** loop
* batch edit with **eppy_json**
* save modified idf files
* run energy plus on eppy
------

### Install eppy

As we already covered in the eppy/geomeppy installation tutorial, you need to install geomeppy by **pip install eppy** in your **conda** environment, if you have already done so, skip this step.

pip install eppy

----

### Install EnergyPlus

Next install **EnergyPlus** if you have not done so. 

**EnergyPlus version**: This tutorial will be running with EnergyPlus 9.1.0. But if you are using other versions, don't worry, geomeppy will also work for your EnergyPlus version. Just remember to change the idd path properly to your version, and keep using one version of EnergyPlus in one jupyter launcher. Otherwise you would see an error messege like this:

<img src="images/idderro.png" width='80%'>

Next, get the idd filepath from the EnergyPlus installation folder. This tutorial is on Mac, so the idd filepath would be: '/Applications/EnergyPlus-9-1-0/Energy+.idd'. 
On windows the path would usually be: 'C:/EnergyPlusV9-1-0', and on Linux '/usr/local/EnergyPlus-9-1-0'.

### Run eppy:

Next, we are going to import idf file to **eppy**. 

Pick an **idf file** that you wish you access. This tutorial uses the 'Minimal.idf' file from the energy plus example folder: '/Applications/EnergyPlus-9-1-0/ExampleFiles/Minimal.idf'.

With these ready, in the python 3 environment, type: 

In [ ]:
# set up file path for idf in eppy
from eppy.modeleditor import IDF
IDF.setiddname('/Applications/EnergyPlus-9-1-0/Energy+.idd')
idf= IDF('/Applications/EnergyPlus-9-1-0/ExampleFiles/VentilatedSlab.idf')

If you are dealing with multiple idf files, you can name them with name that you perfer, such as idf1, idf2, etc.

Then we need to add a a weather file to our IDF object. This tutorial uses 'USA_CO_Golden-NREL.724666_TMY3.epw'

In [ ]:
idf.epw = 'USA_CO_Golden-NREL.724666_TMY3.epw'

### Edit Material

Now we  could edit material with eppy.

Function **.idfobjects['MATERIAL']** shows all materials in this idf file.

You can also view other idf objects, such as .idfobjects['BUILDING'], .idfobjects['construction'.upper()], etc.

Note: All CAPs inside ['']

In [ ]:
# view all materials from a idf file, note ALL CAP 'MATERIAL'
materials=idf.idfobjects['MATERIAL']
materials

<img src="./images/allmaterial.png" width="30%">

materials that we get from **idf.idfobjects['MATERIAL']** is a **list**, We can check the number of materials in this idf file by using **len(materials)**

In [ ]:
# check numbers of materials in the idf file
len(materials)

We've got  17 type of materials in this idf file. 

We could view the first three material.

In [ ]:
# show no.1-3 material in the list
print(materials[0:3])

<img src="images/0-3material.png" width='25%'>

We could view only the first material in the list.

In [ ]:
# view 1st material obejct from the material list
material0=idf.idfobjects['MATERIAL'][0]
material0

<img src="images/material.png" width='40%'>

We could modify all the fields in this material

In [ ]:
# modifying material1
material0.Name='WD10_CHANGED'
material0.Specific_Heat=2000.0
material0.Thickness=0.2
material0.Conductivity=0.5
material0.Density=400
material0.Specific_Heat=1000
material0

<img src="images/material_edited.png" width='40%'>

We could create a new material to the idf file with fields specified

In [ ]:
# create a new material
# !!! since idf.newidfobject is a function to idf file, avoid duplication run
newmaterial=idf.newidfobject('MATERIAL', Name="G01a 19mm gypsum board",
                            Roughness="MediumSmooth",
                            Thickness=0.019,
                            Conductivity=0.16,
                            Density=800,
                            Specific_Heat=1090)
newmaterial

<img src="images/new_material.png" width=30%>

Now let's check how many material is in the idf file. It's 18!

In [31]:
# check whether the newmaterial is added to the material list
len(materials)

18

The added material would be put the last to the list. We can view the last material in the list by:

In [ ]:
# view the last material in the list
idf.idfobjects['MATERIAL'][-1]

<img src="images/new_material_view.png" width='40%'>

Yes, it is the material we just created. Now we could even duplicate this material by using **.copyidfobject()**

In [ ]:
# duplicate material 
newmaterial_copy=idf.copyidfobject(newmaterial)
newmaterial_copy

Let's check the number of material in the idf file again, it's added to 19 now

In [34]:
# now there should be 18+1 materials in the list
len(materials)

19

If there are material you don't want, you can delete it from this idf file in two ways:
1. **.popidfobject('MATERIAL',-1)** in side () you put in list name and index of the item to be removed, popidfobject shows the removed object
2. **.removeidfobject()** inside() you put the item to be removed
Note: these are all functions to the idf file, avoid duplicate run, it will delete things you don't  want to delete

In [ ]:
# remove material
# pop, show the removed material
# pop: popidfobject -> give it the idf key: “MATERIAL”, and the index number
# IDF.popidfobject(key, index)
idf.popidfobject('MATERIAL',-1)

<img src="images/new_material_view.png" width='40%'>

In [ ]:
# now there should be 19-1 materials in the list
len(materials)

In [ ]:
# you  could also use removeidfobject -> give it the idf object to be deleted
material_toremove=idf.idfobjects['MATERIAL'][-1]
idf.removeidfobject(material_toremove)

In [ ]:
# now there should be 18-1 materials in the list
len(materials)

After running two types of deleting methods, we can see the number of material get to (19-2)=17

To view all material's name without their specs, we could:

In [ ]:
# print all names of materials
for material in materials:
    print (material.Name)

<img src="images/material_names.png" width='30%'>

We could use python's **for** logic here to show only material thicker than 0.1

In [ ]:
# show all material name of which thickness >0.1
[material.Name for material in materials if material.Thickness > 0.1]

<img src="images/thick_material.png" width='30%'>

## construction

After talking about material, let's talk about construction, which manage the combination of material 

We could view all construction by using same function **.idfobjects['']** as viewing all material:

In [ ]:
allconstruction=idf.idfobjects['Construction'.upper()]
allconstruction

<img src="images/allconstruction.png" width="30%">

Similarly, we could check the number of  construction by using **len()**

In [ ]:
len(allconstruction)

Lets take a close look at the wall construction in this idf

In [ ]:
construction_wall=allconstruction[1]
construction_wall

<img src="images/wall-1.png" width="30%">

We could duplicate this  construction

In [ ]:
# copy the construction WALL-1
construction_wall_duplicate=idf.copyidfobject(construction_wall)

In [ ]:
len(allconstruction)

We could change the fields of this duplicated construction

In [ ]:
# edit the duplicated WALL-1
construction_wall_duplicate.Name='WALL-2'
construction_wall_duplicate.Outside_Layer='WD01'
construction_wall_duplicate

<img src="images/wall-2.png" width="30%">

In geomeppy, we could also  import construction from other idf files, that's covered in the geomeppy tutorial.

## building surfaces

With material and construction covered, let's take a look at tha building surfaces

Similarly, we could view all building surfaces in this idf

In [ ]:
# view all Building Surfaces
allsurfaces=idf.idfobjects['BuildingSurface:Detailed'.upper()]
allsurfaces

Let's take a look at the first surface in the list

<img src="images/bsurface.png" width="30%">

In [ ]:
surface0=allsurfaces[0]
surface0

<img src="images/surface0.png" width="40%">

View can see the first surface is a WALL, it uses WALL-1 construction. We could change the construction to our newly defined construction above, WALL-2

In [ ]:
# change construction of a surface
surface0.Construction_Name='WALL-2'
surface0

<img src="images/surface0-wall2.png" width="40%">

## building

After talking through material, construction, buildingsurfaces in eppy, lets talk  about building.

Similarly, you can view building in this idf file.

In [ ]:
building=idf.idfobjects['BUILDING']
building

<img src="images/building.png" width="40%">

You can see the type of **idf.idfobjects['BUILDING']** is a MSequence, although there is only one building, it is wraped  in MSequence structure. To apply the **.fieldnames** function that we are going to use next, we need to convert MSequence to EpBunch by simply calling out its index **idfobjects['BUILDING'][0]**

In [ ]:
print(type(building))

<img src="images/list.png" width="40%">

In [ ]:
building=idf.idfobjects['BUILDING'][0]
building

<img src="images/building_1.png" width="40%">

In [ ]:
print(type(building))

<img src="images/epbunch.png" width="40%">

Now we've get EpBunch, we could apply **.fieldnames** to see all fieldnames in the building

In [ ]:
# get all field names of the building idf object
building.fieldnames

<img src="images/allkeys.png" width="40%">

We could get range of any of the field by using **.getrange('')**, here let's try 'Loads_Convergence_Tolerance_Value'

In [ ]:
building.getrange("Loads_Convergence_Tolerance_Value")

<img src="images/range.png" width="20%">

We could assign a value to this field. Obviously, it's out of its range, let's see what will happen. 

In [ ]:
building.Loads_Convergence_Tolerance_Value=0.6

By using **.checkrange("")**, it will warn us an RangeError messege telling us the value is out of range.

In [ ]:
building.checkrange("Loads_Convergence_Tolerance_Value")

<img src="images/range_error.png" width="50%">

To show and check value in more than one fields, we could use **for** logic:

In [ ]:
# get all field names AND value of the building idf object
for fieldname in building.fieldnames:
    print ("%s = %s" % (fieldname, building[fieldname]))

<img src="images/fieldname.png" width="40%">

We could use try and except logic inside for loop to not only show value but also check wether the value is within the range

In [ ]:
# mark out of range value
from eppy.bunch_subclass import RangeError
for fieldname in building.fieldnames:
    try:
        building.checkrange(fieldname)
        print ("%s = %s #-in range" % (fieldname, building[fieldname]))
    except RangeError:
        print ("%s = %s #-****OUT OF RANGE****" % (fieldname, building[fieldname]))

<img src="images/fieldname_range.png" width="40%">

Let's change the value of load converangence tolerance back to 0.5, otherwise it will raise an error in running the idf file.

<img src="images/value_wrong.png" width="60%">

In [ ]:
building.Loads_Convergence_Tolerance_Value=0.5

## edit idf with json in eppy

We have mentioned many ways to change fields in idf objects. To push this further, with advantage of python, you could use json_function in eppy to batch edits multiple fields in various idf objects: 

In [ ]:
import eppy.json_functions as json_functions

In [ ]:
## using json to update idf file
json_str = {"idf.VERSION..Version_Identifier":9.2,
            "idf.SIMULATIONCONTROL..Do_Plant_Sizing_Calculation": "No",
            "idf.BUILDING.Building.North_Axis": 52,
            }
json_functions.updateidf(idf,json_str)

You can see json_str is a dictionary that collect all the list commend together into one

In [ ]:
print(type(json_str))

You can view the idf now again and see wether the fields you want to alter have been changed

In [ ]:
idf.printidf()

<img src="images/print_idf.png" width="40%">

## zone

In the end of this tutorial, lets check out zone

Similarly, we could view all zones by **.idfobjects["zone".upper()]**

In [ ]:
#  show all zones
allzone=idf.idfobjects["zone".upper()]
allzone

<img src="images/zones.png" width="40%">

In this selected zone,  we could see its volume by viewing the value in Volume and area by using function **modeleditor.zonearea(idf, zone1.Name)**

In [ ]:
# view first zone in the list 
zone0=allzone[0]
zone0

<img src="images/zone0.png" width="40%">

In [ ]:
# display zone1 volume and area
from eppy import modeleditor
print ("volume = %s" % (zone0.Volume, ))
area1 = modeleditor.zonearea(idf, zone0.Name)
print ('area = %s ' % (area1,))

<img src="images/VA.png" width="40%">

## run idf
After all the alternations have been done, you could run the idf file and save the idf file

In [ ]:
idf.run()
#idf.save()
#idf.saveas()

<img src="images/run.png" width="100%">

## sources

The source documentation of eppy could be found here: https://eppy.readthedocs.io/en/latest/index.html